In [2]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [1]:
import os
import pandas as pd
import re
from pathlib import Path

# --- LINUX PATHS ---
root_dir = Path("/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/inputs/images")
excel_path = Path("/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/inputs/images/MetaData_Bsubt_KD_screen.xlsx")
output_file = Path("/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/inputs/metadata/index.csv")

# --- HELPER FUNCTIONS ---
def normalize_well_id(well):
    """Converts A1 -> A01, A11 -> A11"""
    match = re.match(r"([a-zA-Z])(\d+)", str(well).strip())
    if match:
        letter = match.group(1).upper()
        number = int(match.group(2))
        return f"{letter}{number:02d}"
    return str(well).strip()

def extract_gene(text):
    if pd.isna(text): return "Unknown"
    match = re.search(r'Gene target:\s*([\w-]+)', str(text))
    return match.group(1) if match else "Unknown"

# --- 1. LOAD ALL EXCEL SHEETS ---
print("Loading all Excel sheets...")
# sheet_name=None loads all sheets into a dictionary: {sheet_name: dataframe}
excel_dict = pd.read_excel(excel_path, sheet_name=None)

# Process each sheet into a treatment map
# Final structure: { 'PLATE_T2': {'A01': 'GeneA', 'A02': 'GeneB'}, 'PLATE_T3': {...} }
plate_treatment_maps = {}

for sheet_name, df in excel_dict.items():
    if 'Strain_name' in df.columns and 'Position_microscopy' in df.columns:
        df['Treatment'] = df['Strain_name'].apply(extract_gene)
        # Create a map for THIS specific sheet/plate
        current_map = dict(zip(
            df['Position_microscopy'].apply(normalize_well_id), 
            df['Treatment']
        ))
        plate_treatment_maps[sheet_name.strip()] = current_map

# --- 2. BUILD THE INDEX ---
all_rows = []
file_pattern = re.compile(r'Sample_(?P<well>[A-Z][0-9]+)_XY(?P<site>[0-9]+)_C(?P<chan>[0-9]+)')

print(f"Scanning directory: {root_dir}")

for plate_folder in os.listdir(root_dir):
    if "PLATE" not in plate_folder.upper():
        continue
    
    plate_path = root_dir / plate_folder
    if not plate_path.is_dir(): continue
    
    # Get the specific treatment map for this plate
    # We use .get() in case the folder name doesn't match the sheet name perfectly
    current_plate_map = plate_treatment_maps.get(plate_folder.strip(), {})
    
    if not current_plate_map:
        print(f"Warning: No Excel sheet found matching folder name '{plate_folder}'")
    
    for well_folder in os.listdir(plate_path):
        if not well_folder.startswith("Sample_"):
            continue
            
        well_path = plate_path / well_folder
        raw_well_id = well_folder.replace("Sample_", "")  # "A1"
        lookup_well_id = normalize_well_id(raw_well_id)    # "A01"
        
        sites = {} 
        for filename in os.listdir(well_path):
            if not filename.lower().endswith((".tiff", ".tif")):
                continue
            match = file_pattern.search(filename)
            if match:
                site_num = match.group('site')
                channel_num = f"C{match.group('chan')}"
                if site_num not in sites: sites[site_num] = {}
                sites[site_num][channel_num] = f"{plate_folder}/{well_folder}/{filename}"

        for site_id, channels in sites.items():
            row = {
                "Metadata_Plate": plate_folder,
                "Metadata_Well": raw_well_id,
                "Metadata_Site": int(site_id),
                "Treatment": current_plate_map.get(lookup_well_id, "Unknown"),
                "Replicate": 1 
            }
            row.update(channels)
            all_rows.append(row)

# --- 3. SAVE ---
if not all_rows:
    print("!!! NO IMAGES FOUND !!!")
else:
    df_final = pd.DataFrame(all_rows)
    channel_cols = sorted([c for c in df_final.columns if c.startswith("C")], 
                          key=lambda x: int(x[1:]) if x[1:].isdigit() else 0)
    final_cols = ["Metadata_Plate", "Metadata_Well", "Metadata_Site", "Treatment", "Replicate"] + channel_cols
    
    output_file.parent.mkdir(parents=True, exist_ok=True)
    df_final[final_cols].to_csv(output_file, index=False)
    
    unknown_count = len(df_final[df_final["Treatment"] == "Unknown"])
    print(f"DONE! Found {len(df_final)} sites.")
    if unknown_count > 0:
        print(f"Warning: {unknown_count} sites have 'Unknown' treatment. Check if sheet names match folder names exactly.")

Loading all Excel sheets...
Scanning directory: /media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/inputs/images
DONE! Found 10 sites.


/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home

In [12]:
import os
import pandas as pd
import re
from pathlib import Path

# --- CONFIGURATION ---
root_dir = Path("/media/arnout/Elements1/Thesis/Screen crispri quality checked")
excel_path = Path("/media/arnout/Elements1/Thesis/Screen crispri quality checked/MetaData_Bsubt_KD_screen.xlsx")
output_file = Path("/media/arnout/Elements1/Thesis/project_neighboursT20/inputs/metadata/index5.csv")            #als ik hier index2 schreef werkte het plots wel , anders nam die de oude met t1t2

# --- PLATE SELECTION ---
# Only plates in this list will be added to your index.csv
#PLATES_TO_INCLUDE = ["PLATE1_T0", "PLATE1_T1"] 
PLATES_TO_INCLUDE = ["PLATE2_T0", "PLATE2_T1", "PLATE2_T2" ] 

# --- HELPER FUNCTIONS ---
def normalize_well_id(well):
    """Converts A1 -> A01, A11 -> A11"""
    match = re.match(r"([a-zA-Z])(\d+)", str(well).strip())
    if match:
        letter = match.group(1).upper()
        number = int(match.group(2))
        return f"{letter}{number:02d}"
    return str(well).strip()

def extract_gene(text):
    if pd.isna(text): return "Unknown"
    match = re.search(r'Gene target:\s*([\w-]+)', str(text))
    return match.group(1) if match else "Unknown"

# --- 1. LOAD ALL EXCEL SHEETS ---
print("Loading all Excel sheets...")
excel_dict = pd.read_excel(excel_path, sheet_name=None)

plate_treatment_maps = {}
for sheet_name, df in excel_dict.items():
    s_name = sheet_name.strip()
    if s_name in PLATES_TO_INCLUDE:  # Only process relevant sheets
        if 'Strain_name' in df.columns and 'Position_microscopy' in df.columns:
            df['Treatment'] = df['Strain_name'].apply(extract_gene)
            current_map = dict(zip(
                df['Position_microscopy'].apply(normalize_well_id), 
                df['Treatment']
            ))
            plate_treatment_maps[s_name] = current_map

# --- 2. BUILD THE INDEX ---
all_rows = []
file_pattern = re.compile(r'Sample_(?P<well>[A-Z][0-9]+)_XY(?P<site>[0-9]+)_C(?P<chan>[0-9]+)')

print(f"Scanning for selected plates in: {root_dir}")

for plate_folder in PLATES_TO_INCLUDE:
    plate_path = root_dir / plate_folder
    
    if not plate_path.exists():
        print(f"Skipping {plate_folder}: Folder not found on disk.")
        continue
    
    print(f"Processing {plate_folder}...")
    current_plate_map = plate_treatment_maps.get(plate_folder, {})
    
    if not current_plate_map:
        print(f"Warning: No Excel sheet found for '{plate_folder}'")
    
    for well_folder in os.listdir(plate_path):
        if not well_folder.startswith("Sample_"):
            continue
            
        well_path = plate_path / well_folder
        raw_well_id = well_folder.replace("Sample_", "")
        lookup_well_id = normalize_well_id(raw_well_id)
        
        sites = {} 
        for filename in os.listdir(well_path):
            if not filename.lower().endswith((".tiff", ".tif")):
                continue
            match = file_pattern.search(filename)
            if match:
                site_num = match.group('site')
                channel_num = f"C{match.group('chan')}"
                if site_num not in sites: sites[site_num] = {}
                sites[site_num][channel_num] = f"{plate_folder}/{well_folder}/{filename}"

        for site_id, channels in sites.items():
            row = {
                "Metadata_Plate": plate_folder,
                "Metadata_Well": raw_well_id,
                "Metadata_Site": int(site_id),
                "Treatment": current_plate_map.get(lookup_well_id, "Unknown"),
                "Replicate": 1 
            }
            row.update(channels)
            all_rows.append(row)

# --- 3. SAVE ---
if not all_rows:
    print("!!! NO IMAGES FOUND FOR THE SELECTED PLATES !!!")
else:
    df_final = pd.DataFrame(all_rows)
    # Filter columns to only include those present in the data
    existing_channels = sorted([c for c in df_final.columns if c.startswith("C")], 
                               key=lambda x: int(x[1:]) if x[1:].isdigit() else 0)
    
    final_cols = ["Metadata_Plate", "Metadata_Well", "Metadata_Site", "Treatment", "Replicate"] + existing_channels
    
    output_file.parent.mkdir(parents=True, exist_ok=True)
    df_final[final_cols].to_csv(output_file, index=False)
    
    print(f"\nSUCCESS! index.csv created with {len(df_final)} sites from {len(df_final['Metadata_Plate'].unique())} plates.")

Loading all Excel sheets...
Scanning for selected plates in: /media/arnout/Elements1/Thesis/Screen crispri quality checked
Processing PLATE2_T0...
Processing PLATE2_T1...
Processing PLATE2_T2...


/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home


SUCCESS! index.csv created with 3977 sites from 3 plates.


In [ ]:
#met emptyrowcleaning

In [ ]:
import os
import pandas as pd
import re
from pathlib import Path

# --- CONFIGURATION ---
root_dir = Path("/media/arnout/Elements1/Thesis/Screen crispri quality checked")
excel_path = Path("/media/arnout/Elements1/Thesis/Screen crispri quality checked/MetaData_Bsubt_KD_screen.xlsx")
output_file = Path("/media/arnout/Elements1/Thesis/project_neighboursT20/inputs/metadata/index.csv")

# --- PLATE SELECTION ---
PLATES_TO_INCLUDE = ["PLATE2_T0", "PLATE2_T1", "PLATE2_T2"] 

# --- HELPER FUNCTIONS ---
def normalize_well_id(well):
    """Converts A1 -> A01, A11 -> A11"""
    match = re.match(r"([a-zA-Z])(\d+)", str(well).strip())
    if match:
        letter = match.group(1).upper()
        number = int(match.group(2))
        return f"{letter}{number:02d}"
    return str(well).strip()

def extract_gene(text):
    if pd.isna(text) or str(text).strip() == "": return "Unknown"
    match = re.search(r'Gene target:\s*([\w-]+)', str(text))
    return match.group(1) if match else "Unknown"

# --- 1. LOAD ALL EXCEL SHEETS ---
print("Loading all Excel sheets...")
excel_dict = pd.read_excel(excel_path, sheet_name=None)

plate_treatment_maps = {}
for sheet_name, df in excel_dict.items():
    s_name = sheet_name.strip()
    if s_name in PLATES_TO_INCLUDE:
        if 'Strain_name' in df.columns and 'Position_microscopy' in df.columns:
            df['Treatment'] = df['Strain_name'].apply(extract_gene)
            current_map = dict(zip(
                df['Position_microscopy'].apply(normalize_well_id), 
                df['Treatment']
            ))
            plate_treatment_maps[s_name] = current_map

# --- 2. BUILD THE INDEX ---
all_rows = []
file_pattern = re.compile(r'Sample_(?P<well>[A-Z][0-9]+)_XY(?P<site>[0-9]+)_C(?P<chan>[0-9]+)')

print(f"Scanning for selected plates in: {root_dir}")

for plate_folder in PLATES_TO_INCLUDE:
    plate_path = root_dir / plate_folder
    
    if not plate_path.exists():
        print(f"Skipping {plate_folder}: Folder not found on disk.")
        continue
    
    print(f"Processing {plate_folder}...")
    current_plate_map = plate_treatment_maps.get(plate_folder, {})
    
    if not current_plate_map:
        print(f"Warning: No Excel sheet found for '{plate_folder}'")
    
    for well_folder in os.listdir(plate_path):
        if not well_folder.startswith("Sample_"):
            continue
            
        well_path = plate_path / well_folder
        raw_well_id = well_folder.replace("Sample_", "")
        lookup_well_id = normalize_well_id(raw_well_id)
        
        sites = {} 
        for filename in os.listdir(well_path):
            if not filename.lower().endswith((".tiff", ".tif")):
                continue
            match = file_pattern.search(filename)
            if match:
                site_num = match.group('site')
                channel_num = f"C{match.group('chan')}"
                if site_num not in sites: sites[site_num] = {}
                sites[site_num][channel_num] = f"{plate_folder}/{well_folder}/{filename}"

        for site_id, channels in sites.items():
            row = {
                "Metadata_Plate": plate_folder,
                "Metadata_Well": raw_well_id,
                "Metadata_Site": int(site_id),
                "Treatment": current_plate_map.get(lookup_well_id, "Unknown"),
                "Replicate": 1 
            }
            row.update(channels)
            all_rows.append(row)

# --- 3. CLEANING AND SAVING ---
if not all_rows:
    print("!!! NO IMAGES FOUND FOR THE SELECTED PLATES !!!")
else:
    df_final = pd.DataFrame(all_rows)
    
    # Organize columns
    existing_channels = sorted([c for c in df_final.columns if c.startswith("C")], 
                               key=lambda x: int(x[1:]) if x[1:].isdigit() else 0)
    final_cols = ["Metadata_Plate", "Metadata_Well", "Metadata_Site", "Treatment", "Replicate"] + existing_channels
    df_final = df_final[final_cols]

    # A. Report on "Unknown" Treatments
    unknown_mask = df_final['Treatment'] == "Unknown"
    num_unknown = unknown_mask.sum()
    if num_unknown > 0:
        print(f"\n[NOTICE] {num_unknown} sites were assigned 'Unknown' treatment (missing in Excel).")

    # B. Convert truly empty strings/whitespace to NaN so they can be dropped
    df_final = df_final.replace(r'^\s*$', pd.NA, regex=True)

    # C. Identify rows with ANY missing value (NaN or empty)
    initial_count = len(df_final)
    rows_with_nan = df_final.isnull().any(axis=1)

    if rows_with_nan.any():
        print(f"\n[CLEANING] Removing {rows_with_nan.sum()} rows containing empty/missing fields:")
        # Print details of what is being removed
        print(df_final[rows_with_nan][["Metadata_Plate", "Metadata_Well", "Metadata_Site"]].to_string(index=False))
        
        # Keep only the rows that are NOT missing data
        df_final = df_final[~rows_with_nan].copy()
    else:
        print("\n[CLEANING] No empty fields found. All rows kept.")

    # D. Final Save
    output_file.parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(output_file, index=False)
    
    print(f"\nSUCCESS! index.csv created at: {output_file}")
    print(f"Final summary: {len(df_final)} sites kept | {initial_count - len(df_final)} rows removed.")

Loading all Excel sheets...
Scanning for selected plates in: /media/arnout/Elements1/Thesis/Screen crispri quality checked
Processing PLATE2_T0...
Processing PLATE2_T1...
Processing PLATE2_T2...

[CLEANING] Removing 16 rows containing empty/missing fields:


/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home

Metadata_Plate Metadata_Well  Metadata_Site
     PLATE2_T0            D9              0
     PLATE2_T0            G7              0
     PLATE2_T0            B2              3
     PLATE2_T0            B4             13
     PLATE2_T0            B5              0
     PLATE2_T0            C3              0
     PLATE2_T0            C5              0
     PLATE2_T0            C6              0
     PLATE2_T0           D10              0
     PLATE2_T0            D2              1
     PLATE2_T0            D5              0
     PLATE2_T0            D6              0
     PLATE2_T1            E4              0
     PLATE2_T1           C11              0
     PLATE2_T1            D2              0
     PLATE2_T2            D7              5

SUCCESS! index.csv created at: /media/arnout/Elements1/Thesis/project_neighboursT20/inputs/metadata/index5.csv
Final summary: 3961 sites kept | 16 rows removed.
